In [7]:
from time import perf_counter
import numpy as np,os
import pandas as pd
from collections import defaultdict
from sklearn.utils import shuffle
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RepeatedStratifiedKFold, StratifiedKFold
from imblearn.under_sampling import InstanceHardnessThreshold, RandomUnderSampler, EditedNearestNeighbours
from imblearn.pipeline import Pipeline

from sklearn.metrics import matthews_corrcoef, precision_recall_curve, auc, accuracy_score, precision_score, recall_score
import seaborn as sns,matplotlib.pyplot as plt

from library.configs import CLFS, CV, SCORERS
from library.utils import evaluate, read_data

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
DATASETS = ['groovy-1_5_7.csv','jruby-1.4.0.csv','lucene-2.9.0.csv','jruby-1.7.0.preview1.csv','groovy-1_6_BETA_1.csv',
        'derby-10.2.1.6.csv','wicket-1.5.3.csv','camel-2.9.0.csv','camel-1.4.0.csv','activemq-5.8.0.csv']
DATASETS = [f for f in os.listdir("JIRA/") if 'csv' in f]
len(DATASETS)

32

In [9]:
from sklearn.base import BaseEstimator, TransformerMixin, ClassifierMixin, clone
class Spyder(BaseEstimator, ClassifierMixin):
    def __init__(self,estimator,K=5):
        self.estimator = estimator
        self.K = K
        
    def sample(self,X,Y,sample_weight):    # SPIDER2, relabel=False and ampl=weak by default
        if sample_weight is None:
            sample_weight = np.ones_like(Y)

        # First step, Remove unsafe samples from majority, 
        enn = EditedNearestNeighbours(sampling_strategy='majority',n_neighbors=5,kind_sel='mode')
        Xs,Ys = enn.fit_resample(X,Y)
        sample_weight = sample_weight[enn.sample_indices_]
            
        #Second, upsample unsafe minority samples
        disagreement = (kDN(Xs,Ys,K=5,weight='uniform') * 5).astype('int')
        agreement = 5 - disagreement

        n = disagreement - agreement  #Number of times to upsample
        n[Ys==0] = 0  #Don't upsample majority-class samples
        n[n<0] = 0

        Xt,Yt,SWt = Xs.copy(),Ys.copy(),sample_weight.copy()
        while n.sum()>0:
            Xtmp, Ytmp, SWtmp = Xs[n>0].copy(),Ys[n>0].copy(),sample_weight[n>0].copy()
            Xt,Yt,SWt = np.concatenate((Xt,Xtmp)),np.concatenate((Yt,Ytmp)),np.concatenate((SWt,SWtmp))
            
            n -= 1
            n[n==-1] = 0
            
        return Xt,Yt,SWt
    
    def fit(self, X, Y,sample_weight=None):
        Xf,Yf,sample_weight = self.sample(X, Y, sample_weight)
#         a,b = np.unique(Y,return_counts=True)[1],np.unique(Yf,return_counts=True)[1]
#         print(a.max()/a.min(),b.max()/b.min(),a,b,Xf.shape,len(Xf)/len(X))
        assert len(np.unique(Yf))==2,"Pos class completely filtered out"
        try:
            self.estimator = self.estimator.fit(Xf, Yf,sample_weight=sample_weight)
        except TypeError as e:
            self.estimator = self.estimator.fit(Xf, Yf)
        return self

    @property
    def classes_(self):
        return self.estimator.classes_

    def predict(self, X):
        return self.estimator.predict(X)
    
    def predict_proba(self, X):
        return self.estimator.predict_proba(X) 

In [10]:
models = {}
for c,clf in CLFS.items():
    models[('Spyder',c)] = Spyder(clf)    
    
models.keys(),len(models)

(dict_keys([('Spyder', 'dt'), ('Spyder', 'lr'), ('Spyder', 'nb'), ('Spyder', 'svm'), ('Spyder', 'knn'), ('Spyder', 'rf')]),
 6)

In [12]:
path = "Spyder.csv"
cols = pd.MultiIndex.from_product([['Spyder'],CLFS.keys(),[f.__name__ for f in SCORERS]],names=['imb','clf','metric'])
df = pd.DataFrame(index=DATASETS,columns=cols)

In [ ]:
%%time
for it,d in enumerate(DATASETS):
    print(it)
    X,y_noisy,y_real = read_data(d,stats=True)
    for k in models:
        print(k)
        sd = perf_counter()
        r = evaluate(models[k],X,y_noisy,y_real,CV,SCORERS)
        for f in r:
            df.loc[d,(k[0],k[1],f)] = r[f].mean()
        print(round(perf_counter()-sd,2),[round(r[f].mean(),3) for f in r])
    print()
    df.to_csv(path)